### Source: 
    
https://octis.readthedocs.io/en/latest/readme.html#topic-models-and-evaluation

In [1]:
from coherence_metrics import Coherence
from coherence_metrics import WECoherencePairwise
from coherence_metrics import WECoherenceCentroid
from diversity_metrics import InvertedRBO
import pandas as pd

In [2]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

In [3]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter",
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

C:\Users\basharm\Anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


C:\Users\basharm\Anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [5]:
df_train = pd.read_csv('data\\general_sentiment_small\\train_tweet.csv')
df_train

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [6]:
df_train = df_train[['label', 'tweet']]

In [7]:
df_train.columns = ['label', 'text']

In [8]:
df_train

,label,text
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [9]:
#count = 1
for idx in df_train.index:
    sent = df_train.loc[idx,'text']
    sent = sent.replace('‘', '\'').replace('’', '\'').replace('“', '"').replace('”', '"')
    #print(sent)
    sent = ' '.join(text_processor.pre_process_doc(sent))
    #print(sent)
    df_train.loc[idx,'text'] = sent
    if idx%1000==0:
        print('current index', idx)
        
df_train

C:\Users\basharm\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


current index 0
current index 1000
current index 2000
current index 3000
current index 4000
current index 5000
current index 6000
current index 7000
current index 8000
current index 9000
current index 10000
current index 11000
current index 12000
current index 13000
current index 14000
current index 15000
current index 16000
current index 17000
current index 18000
current index 19000
current index 20000
current index 21000
current index 22000
current index 23000
current index 24000
current index 25000
current index 26000
current index 27000
current index 28000
current index 29000
current index 30000
current index 31000


,label,text
0,0,<user> when a father is dysfunctional and is s...
1,0,<user> <user> thanks for <hashtag> lyft </hash...
2,0,bihday your majesty
3,0,<hashtag> model </hashtag> i love u take with ...
4,0,factsguide : society now <hashtag> motivation ...
...,...,...
31957,0,ate <user> isz that you <elongated> ? ð    ...
31958,0,to see nina turner on the airwaves trying to w...
31959,0,listening to sad songs on a monday morning otw...
31960,1,<user> <hashtag> sikh </hashtag> <hashtag> tem...


In [10]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import re

def remove_auxilary(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.replace('user', '').replace('hashtag', '').replace('allcaps', '').replace('url', '')
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    return tokens_without_sw

text = '<user> wow that hoes aint shit , you deserve better dan that ugly whore brother'
tokens_without_auxi = remove_auxilary(text)
print(tokens_without_auxi)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\basharm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\basharm\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['wow', 'hoes', 'aint', 'shit', 'deserve', 'better', 'ugly', 'whore', 'brother']


In [11]:
text_wa = []
for idx in df_train.index:
    sent = df_train.loc[idx,'text']
    sent = remove_auxilary(sent)
    sent = ' '.join(sent)
    text_wa.append(sent)
    if idx%1000==0:
        print('current index', idx)
        
df_train['text_wa'] = text_wa
df_train

current index 0
current index 1000
current index 2000
current index 3000
current index 4000
current index 5000
current index 6000
current index 7000
current index 8000
current index 9000
current index 10000
current index 11000
current index 12000
current index 13000
current index 14000
current index 15000
current index 16000
current index 17000
current index 18000
current index 19000
current index 20000
current index 21000
current index 22000
current index 23000
current index 24000
current index 25000
current index 26000
current index 27000
current index 28000
current index 29000
current index 30000
current index 31000


,label,text,text_wa
0,0,<user> when a father is dysfunctional and is s...,father dysfunctional selfish drags kids dysfun...
1,0,<user> <user> thanks for <hashtag> lyft </hash...,thanks lyft credit use cause offer wheelchair ...
2,0,bihday your majesty,bihday majesty
3,0,<hashtag> model </hashtag> i love u take with ...,model love time urð repeated ð ð ð ð ð ð ð
4,0,factsguide : society now <hashtag> motivation ...,factsguide society motivation
...,...,...,...
31957,0,ate <user> isz that you <elongated> ? ð    ...,ate isz elongated ð ð ð ð ð ð ð ð ð â ï
31958,0,to see nina turner on the airwaves trying to w...,see nina turner airwaves trying wrap mantle ge...
31959,0,listening to sad songs on a monday morning otw...,listening sad songs monday morning otw work sad
31960,1,<user> <hashtag> sikh </hashtag> <hashtag> tem...,sikh temple vandalised calgary w condemns act


In [12]:
texts = [text.split() for text in df_train['text_wa'].tolist()]
texts

[['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'run'],
 ['thanks',
  'lyft',
  'credit',
  'use',
  'cause',
  'offer',
  'wheelchair',
  'vans',
  'pdx',
  'disapointed',
  'get',
  'thanked'],
 ['bihday', 'majesty'],
 ['model',
  'love',
  'time',
  'urð',
  'repeated',
  'ð',
  'ð',
  'ð',
  'ð',
  'ð',
  'ð',
  'ð'],
 ['factsguide', 'society', 'motivation'],
 ['number',
  'number',
  'huge',
  'fan',
  'fare',
  'big',
  'talking',
  'leave',
  'chaos',
  'pay',
  'disputes',
  'get',
  'show',
  'go'],
 ['camping', 'tomorrow', 'dannyâ'],
 ['next',
  'school',
  'year',
  'year',
  'exams',
  'ð',
  'think',
  'ð',
  'school',
  'exams',
  'hate',
  'imagine',
  'actors',
  'life',
  'revolution',
  'school',
  'girl'],
 ['repeated',
  'love',
  'land',
  'repeated',
  'cavs',
  'champions',
  'cleveland',
  'cleveland',
  'cavaliers',
  'â'],
 ['welcome', 'gr8'],
 ['â',
  'ireland',
  'consumer',
  'price',
  'index',
  'mom',
  'climbed',
  'previous',
  

In [13]:
import numpy as np
np.random.seed(2018)

### LDA

In [14]:
# Topic modelling LDA
import gensim
from gensim import corpora, models

In [15]:
# Create a dictionary from "pp_paragraphs" containing the number of times a word appears in the training set.
dictionary = gensim.corpora.Dictionary(texts)

In [16]:
bow_corpus = [dictionary.doc2bow(para) for para in texts]
bow_corpus[3410]

[(21, 1), (574, 1), (1211, 1), (4594, 1), (5114, 1), (8098, 1)]

In [ ]:
# Convert the paragraphs into TF-IDF representation
# tfidf = models.TfidfModel(bow_corpus)
# corpus_tfidf = tfidf[bow_corpus]
   
# print(corpus_tfidf[3410])

In [17]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=10, workers=2)

In [ ]:
# # For each topic, explore the words occuring in that topic and its relative weight.
# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))

In [18]:
lda_topics=lda_model.show_topics()

twords=[]
for topic,word in lda_topics:
    twords.append(re.sub('[^A-Za-z ]+', '', word).split())
print(twords)

[['thankful', 'good', 'insta', 'positive', 'love', 'follow', 'smile', 'day'], ['number', 'day', 'repeated', 'happy', 'bihday', 'new', 'bih', 'life', 'home'], ['love', 'repeated', 'time', 'day', 'friday', 'happy', 'weekend'], ['need', 'repeated', 'work', 'sad', 'happy', 'people', 'rip', 'right', 'city', 'never'], ['positive', 'affirmation', 'like', 'get', 'looking', 'today', 'free', 'repeated', 'look', 'job'], ['bear', 'polar', 'angry', 'racing', 'living', 'people', 'book', 'climb', 'lot', 'make'], ['blog', 'number', 'gold', 'music', 'silver', 'forex', 'video', 'percent'], ['repeated', 'day', 'number', 'father', 'see', 'fathers', 'first', 'time', 'wait', 'happy'], ['repeated', 'healthy', 'bull', 'heal', 'ways', 'tweets', 'whatever', 'healing', 'delete', 'found'], ['like', 'orlando', 'love', 'trump', 'life', 'repeated', 'people', 'number', 'us']]


In [19]:
lda_model_output = {'topics':twords}

lda_model_output

{'topics': [['thankful',
   'good',
   'insta',
   'positive',
   'love',
   'follow',
   'smile',
   'day'],
  ['number',
   'day',
   'repeated',
   'happy',
   'bihday',
   'new',
   'bih',
   'life',
   'home'],
  ['love', 'repeated', 'time', 'day', 'friday', 'happy', 'weekend'],
  ['need',
   'repeated',
   'work',
   'sad',
   'happy',
   'people',
   'rip',
   'right',
   'city',
   'never'],
  ['positive',
   'affirmation',
   'like',
   'get',
   'looking',
   'today',
   'free',
   'repeated',
   'look',
   'job'],
  ['bear',
   'polar',
   'angry',
   'racing',
   'living',
   'people',
   'book',
   'climb',
   'lot',
   'make'],
  ['blog', 'number', 'gold', 'music', 'silver', 'forex', 'video', 'percent'],
  ['repeated',
   'day',
   'number',
   'father',
   'see',
   'fathers',
   'first',
   'time',
   'wait',
   'happy'],
  ['repeated',
   'healthy',
   'bull',
   'heal',
   'ways',
   'tweets',
   'whatever',
   'healing',
   'delete',
   'found'],
  ['like',
   'orlan

In [20]:
ch = Coherence(texts = texts, topk=5, measure='c_npmi') # taw: measures how related the top-10 words of a topic are to each other, considering the words’ empirical frequency in the original corpus.
taw = ch.score(lda_model_output)
taw

0.03546172222757564

In [21]:
ch = Coherence(texts = texts, topk=5, measure='u_mass') # taw: measures how related the top-10 words of a topic are to each other, considering the words’ empirical frequency in the original corpus.
taw = ch.score(lda_model_output)
taw

-4.6372045156475235

In [22]:
wechp = WECoherencePairwise(topk=5) # alpha: provides an additional measure of how similar the words in a topic are.
alpha = wechp.score(lda_model_output)
alpha

0.47180754184722906

In [23]:
invRBO = InvertedRBO(topk=5, weight=0.9) #rho: evaluates how diverse the topics generated by a single model are.
rho = invRBO.score(lda_model_output)
rho

0.9083307777777778

In [24]:
report = '\nLDA Model Results\n-----------------------------\n'
report += 'Taw\tAlpha\tRho\tNoOfTopics\tNoOfWords\n'
report += '{}\t{}\t{}\t{}\t{}\n'.format(taw, alpha, rho, 5, 5)
print(report)


LDA Model Results
-----------------------------
Taw	Alpha	Rho	NoOfTopics	NoOfWords
-4.6372045156475235	0.47180754184722906	0.9083307777777778	5	5



In [25]:
with open('general_sentiment_small_data_report.txt', 'a+', encoding='utf8') as FO:
    FO.write(report)

### NMF

In [26]:
from gensim.models import Nmf
from gensim.corpora.dictionary import Dictionary

In [27]:
#from gensim.test.utils import common_texts
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
nmf = Nmf(corpus, id2word=dictionary, num_topics=10)

In [28]:
twords = []
for i,j in nmf.show_topics(num_topics=10, num_words=10):
    j = re.sub("(\+ )?\-?\d*\.\d+\*?","",j)
    j = re.sub('"',"",j)
    twords.append(j.split(" "))
    
nmf_model_output = {'topics':twords}
nmf_model_output

{'topics': [['new',
   'number',
   'bear',
   'polar',
   'climb',
   'racing',
   'get',
   'angry',
   'living',
   'cold'],
  ['happy',
   'good',
   'life',
   'father',
   'day',
   'love',
   'insta',
   'follow',
   'repeated',
   'fathers'],
  ['repeated',
   'healthy',
   'heal',
   'ways',
   'time',
   'elongated',
   'healing',
   'ó¾',
   'best',
   'sad'],
  ['number',
   'thankful',
   'positive',
   'â',
   'ï',
   'blog',
   'gold',
   'silver',
   'forex',
   'people'],
  ['ð',
   'time',
   'model',
   'urð',
   'love',
   'repeated',
   'ï',
   '¼ð',
   '½ð',
   'elongated'],
  ['ã', 'â', '¼ã', 'æ', '³ã', 'ç', '¹ã', 'ªã', 'like', '¼'],
  ['love',
   'number',
   'like',
   'bull',
   'life',
   'work',
   'way',
   'tweets',
   'time',
   'family'],
  ['â', 'ï', 'life', 'love', 'dj', 'smile', 'insta', 'like', 'good', 'blog'],
  ['ð', 'â', 'love', 'time', 'model', 'urð', 'ï', 'repeated', '¼ð', 'good'],
  ['day',
   'fathers',
   'father',
   'bih',
   'today',
   'd

In [29]:
ch = Coherence(texts = texts, topk=10, measure='c_npmi') # taw: measures how related the top-10 words of a topic are to each other, considering the words’ empirical frequency in the original corpus.
taw = ch.score(nmf_model_output)
taw

0.008467024623945097

In [30]:
ch = Coherence(texts = texts, topk=10, measure='u_mass') # taw: measures how related the top-10 words of a topic are to each other, considering the words’ empirical frequency in the original corpus.
taw = ch.score(nmf_model_output)
taw

-5.538205598334058

In [31]:
wechp = WECoherencePairwise(topk=10) # alpha: provides an additional measure of how similar the words in a topic are.
alpha = wechp.score(nmf_model_output)
alpha

0.40954413943820533

In [32]:
invRBO = InvertedRBO(topk=10, weight=0.9) #rho: evaluates how diverse the topics generated by a single model are.
rho = invRBO.score(nmf_model_output)
rho

0.8956590591620635

In [33]:
report = '\nNMF Model Results\n-----------------------------\n'
report += 'Taw\tAlpha\tRho\tNoOfTopics\tNoOfWords\n'
report += '{}\t{}\t{}\t{}\t{}\n'.format(taw, alpha, rho, 10, 10)
print(report)


NMF Model Results
-----------------------------
Taw	Alpha	Rho	NoOfTopics	NoOfWords
-5.538205598334058	0.40954413943820533	0.8956590591620635	10	10



In [34]:
with open('general_sentiment_small_data_report.txt', 'a+', encoding='utf8') as FO:
    FO.write(report)